In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

import keras
from keras.models import Sequential,model_from_json
from keras.layers import Dense
from keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet
from keras.applications.xception import Xception
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [2]:
PATH ='dataset/'

In [3]:
train = pd.read_csv(f'{PATH}train.csv')
test = pd.read_csv(f'{PATH}test.csv')

In [4]:
def split_data(tr_data,n):
    return tr_data.iloc[:len(tr_data)-n],tr_data.iloc[len(tr_data)-n:]

In [5]:
label = train['label']
train = train.drop('label',1)

Using TensorFlow backend.


In [7]:
num_classes = 10


In [8]:
n = 12800
X_train,X_valid = split_data(train,n)
Y_train,Y_valid = split_data(label,n)



#one hot encoding
label1 = keras.utils.to_categorical(label, num_classes)
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_valid = keras.utils.to_categorical(Y_valid, num_classes)

In [10]:
# input image dimensions
img_rows, img_cols = 28, 28
X_train = X_train.as_matrix()
X_valid = X_valid.as_matrix()
test = test.as_matrix()
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_valid = X_valid.reshape(X_valid.shape[0], 1, img_rows, img_cols)
    test =  test.reshape(test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_valid = X_valid.reshape(X_valid.shape[0], img_rows, img_cols, 1)
    test = test.reshape(test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_train /= 255
X_valid /= 255
# test /= 255


In [11]:
test = test.astype('float32')
test /= 255

In [12]:
model = Sequential()
model.add(Conv2D(filters = 60, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 40, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 50, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 50, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 25, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Dropout(0.20))


model.add(Flatten())
model.add(Dense(180, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [13]:
batch_size =124
epochs = 30

In [14]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=RMSprop(epsilon=1e-08),
              metrics=['accuracy'])

In [15]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [ ]:
model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = 10, validation_data = (X_valid,Y_valid),
                              verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size)

Epoch 1/10
235/235 [==============================] - 551s 2s/step - loss: 0.6824 - acc: 0.7751 - val_loss: 0.0772 - val_acc: 0.9758
Epoch 2/10
235/235 [==============================] - 554s 2s/step - loss: 0.2088 - acc: 0.9387 - val_loss: 0.0687 - val_acc: 0.9774
Epoch 3/10
235/235 [==============================] - 557s 2s/step - loss: 0.1458 - acc: 0.9571 - val_loss: 0.0504 - val_acc: 0.9861
Epoch 4/10
235/235 [==============================] - 556s 2s/step - loss: 0.1233 - acc: 0.9642 - val_loss: 0.0400 - val_acc: 0.9884
Epoch 5/10
235/235 [==============================] - 560s 2s/step - loss: 0.1032 - acc: 0.9698 - val_loss: 0.0388 - val_acc: 0.9881
Epoch 6/10
235/235 [==============================] - 559s 2s/step - loss: 0.0889 - acc: 0.9747 - val_loss: 0.0338 - val_acc: 0.9894
Epoch 7/10
235/235 [==============================] - 562s 2s/step - loss: 0.0829 - acc: 0.9755 - val_loss: 0.0322 - val_acc: 0.9905
Epoch 8/10
234/235 [============================>.] - ETA: 2s - loss:

In [ ]:
prediction = model.predict(test)

In [ ]:
result = np.argmax(prediction,axis=1)

In [ ]:
np.savetxt('result_DLCNN_50_1.csv',
          np.c_[range(1,len(test) + 1),result],
          delimiter=',',
            header='ImageId,Label',
           comments='',
          fmt='%d')